# Data Analysis and GIS Storytelling

In this notebook we will explore 311 + ACS Data.

_Note: This is a starter / placeholder file for Felix / Leslie to work on._


## Loading Data from Google Drive

Don't edit this. This is just for loading in all the data.


In [1]:
# Load the Drive helper and mount
# from google.colab import drive
# import os

# # This will prompt for authorization.
# drive.mount('/content/drive', force_remount=True)

# PROJECT_PATH = "/content/drive/MyDrive/CY PLAN 101 Group # 18: 311 Service Requests/codebase"

# os.chdir(PROJECT_PATH)

In [2]:
# !cp -r "data" "/content/data"

In [ ]:
!pip install -r ../requirements.txt -q

# Creating Visualizations


In [4]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
import os

In [5]:
# Load 311 service requests and ACS data
sf_tracts_acs = gpd.read_file("../data/processed/sf_tracts_cleaned_2023.gpkg")
serv_req_gdf = gpd.read_parquet("../data/processed/serv_req_cleaned.parquet")

print("311 Service Requests GeoDataFrame:")
display(serv_req_gdf.head())
print("San Francisco Tracts ACS GeoDataFrame:")
display(sf_tracts_acs.head())

311 Service Requests GeoDataFrame:


,caseid,opened,closed,updated,status,status_notes,responsible_agency,category,request_type,request_details,...,source,year,month,dow,hour,request_duration_hrs,geometry,GEOID,is_closed,response_time_cat
0,16753587,2023-05-05 06:11:00,2023-05-05 06:49:39,2023-05-05 06:49:39,Closed,Case Resolved - Pickup Completed.,Recology Abandoned,Street And Sidewalk Cleaning,Bulky Items,Boxed Or Bagged Items,...,Mobile/Open311,2023,5,Friday,6,0.644167,POINT (-122.4201 37.78726),06075012002,True,<1hr
1,15202335,2022-04-11 09:02:00,2023-01-06 09:11:02,2023-01-06 09:11:02,Closed,Case Resolved - Case Is Resolved,Dpt Signshop Queue,Graffiti,Graffiti On Other Enter Additional Details Below,Other Enter Additional Details Below - Offensive,...,Mobile/Open311,2022,4,Monday,9,6480.150556,POINT (-122.41013 37.78244),06075012503,True,>3days
2,17060945,2023-07-20 03:20:00,2023-07-20 09:04:16,2023-07-20 09:04:16,Closed,Case Resolved - Pickup Completed.,Recology Abandoned,Street And Sidewalk Cleaning,Bulky Items,Boxed Or Bagged Items,...,Web,2023,7,Thursday,3,5.737778,POINT (-122.41952 37.74619),06075025300,True,1–24hr
3,17061575,2023-07-20 08:46:00,2023-07-21 08:42:00,2023-07-21 08:42:00,Closed,Case Resolved,Dpw Ops Queue,Street And Sidewalk Cleaning,General Cleaning,Other Loose Garbage,...,Mobile/Open311,2023,7,Thursday,8,23.933333,POINT (-122.40918 37.78372),06075012503,True,1–24hr
4,13438247,2021-02-01 09:46:48,2023-10-30 10:33:54,2023-10-30 10:33:54,Closed,Case Resolved - This Is A Duplicate Of Rfa#215...,Dpw Bsm Queue,Sidewalk Or Curb,Curb Or Curb Ramp Defect,Curb Or Curb Ramp Defect,...,Web,2021,2,Monday,9,24024.785000,POINT (-122.41058 37.74449),06075025200,True,>3days


San Francisco Tracts ACS GeoDataFrame:


,STATEFP,COUNTYFP,TRACTCE,GEOID,GEOIDFQ,NAME_x,NAMELSAD,MTFCC,FUNCSTAT,ALAND,...,county,tract,poverty_rate,bachelors_plus_rate,pct_renter_occupied,share_white,share_black,share_asian,share_hispanic,geometry
0,06,075,010101,06075010101,1400000US06075010101,101.01,Census Tract 101.01,G5020,S,667587,...,075,010101,0.098303,0.784156,0.906766,0.417118,0.071506,0.329361,0.182015,"MULTIPOLYGON (((552539.999 4184361.006, 552510..."
1,06,075,010102,06075010102,1400000US06075010102,101.02,Census Tract 101.02,G5020,S,79556,...,075,010102,0.118663,0.415786,0.667034,0.319308,0.185591,0.441499,0.053602,"MULTIPOLYGON (((551475.86 4184387.011, 551619...."
2,06,075,010201,06075010201,1400000US06075010201,102.01,Census Tract 102.01,G5020,S,186873,...,075,010201,0.060966,0.703587,0.610909,0.730211,0.055533,0.085371,0.128885,"MULTIPOLYGON (((550649.457 4184035.04, 550787...."
3,06,075,010202,06075010202,1400000US06075010202,102.02,Census Tract 102.02,G5020,S,331839,...,075,010202,0.071306,0.816910,0.690192,0.748096,0.036802,0.180203,0.034898,"MULTIPOLYGON (((550463.948 4184841.313, 550466..."
4,06,075,010300,06075010300,1400000US06075010300,103,Census Tract 103,G5020,S,268261,...,075,010300,0.082564,0.738929,0.767149,0.616940,0.000000,0.259290,0.123770,"MULTIPOLYGON (((551199.059 4184447.418, 551301..."


In [6]:
# Create a combined Gdf with both geometries to tie individual requests into Census Tracts

joined_Gdf = gpd.sjoin(serv_req_gdf, sf_tracts_acs, how="left", predicate="within")
joined_Gdf.head()

/tmp/ipykernel_151992/2560436619.py:3: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:26910

  joined_Gdf = gpd.sjoin(serv_req_gdf, sf_tracts_acs, how="left", predicate="within")


,caseid,opened,closed,updated,status,status_notes,responsible_agency,category,request_type,request_details,...,state,county,tract,poverty_rate,bachelors_plus_rate,pct_renter_occupied,share_white,share_black,share_asian,share_hispanic
0,16753587,2023-05-05 06:11:00,2023-05-05 06:49:39,2023-05-05 06:49:39,Closed,Case Resolved - Pickup Completed.,Recology Abandoned,Street And Sidewalk Cleaning,Bulky Items,Boxed Or Bagged Items,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15202335,2022-04-11 09:02:00,2023-01-06 09:11:02,2023-01-06 09:11:02,Closed,Case Resolved - Case Is Resolved,Dpt Signshop Queue,Graffiti,Graffiti On Other Enter Additional Details Below,Other Enter Additional Details Below - Offensive,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,17060945,2023-07-20 03:20:00,2023-07-20 09:04:16,2023-07-20 09:04:16,Closed,Case Resolved - Pickup Completed.,Recology Abandoned,Street And Sidewalk Cleaning,Bulky Items,Boxed Or Bagged Items,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,17061575,2023-07-20 08:46:00,2023-07-21 08:42:00,2023-07-21 08:42:00,Closed,Case Resolved,Dpw Ops Queue,Street And Sidewalk Cleaning,General Cleaning,Other Loose Garbage,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13438247,2021-02-01 09:46:48,2023-10-30 10:33:54,2023-10-30 10:33:54,Closed,Case Resolved - This Is A Duplicate Of Rfa#215...,Dpw Bsm Queue,Sidewalk Or Curb,Curb Or Curb Ramp Defect,Curb Or Curb Ramp Defect,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#Examination of Census Tracts

Interactive Map to Examine Broader Demographic Trends Aross Tracts


In [ ]:
tooltip_acs = [
    "median_household_income",
    "poverty_rate",
    "total_population",
    "median_home_value",
]

acs_discovery_map = sf_tracts_acs.explore(
    column="median_household_income",
    cmap="RdPu",
    tooltip=tooltip_acs,
    legend=True,
    legend_kwds={"caption": "Median Household Income ($)"},
)

title_discovery_map = """
     <h3 align="center" style="font-size:20px"><b>Median Household Income by Census Tract, San Francisco 2023</b></h3>
"""

acs_discovery_map.get_root().html.add_child(folium.Element(title_discovery_map))
acs_discovery_map.save("../figures/acs_discovery_map.html")
acs_discovery_map

FileNotFoundError: [Errno 2] No such file or directory: 'figures/acs_discovery_map.html'

# Types of Requests by Census Tract and Income


Figure 1: street + sidewalk cleaning requests that are opened and closed in 2023 timescale (2023) per capita per tract and associated graphs (such as relationship between requests per capita and income per tract)

Figure 2: Most popular type of request by tract map + stacked bar perhaps?


In [ ]:
# Find the top categories of requests to consolidate smaller categories as "Other"

req_type_counts = joined_Gdf["category"].value_counts()
top_categories = req_type_counts.index[0:5].tolist()
top_categories

In [ ]:
# Recode to just top categories
joined_Gdf["top_categories"] = joined_Gdf["category"].where(
    joined_Gdf["category"].isin(top_categories), other="Other"
)

In [ ]:
requests_counts = (
    joined_Gdf.groupby(["TRACTCE", "top_categories"]).size().unstack(fill_value=0)
)
# Add group by month for some type of time dimension
tract_requests_total = requests_counts.sum(axis=1)

requests_counts_pct = 100 * requests_counts.div(tract_requests_total, axis=0)
top_category = requests_counts.idxmax(axis=1)

requests_Gdf = sf_tracts_acs
requests_Gdf = requests_Gdf.merge(
    top_category.rename("top request category"), left_on="TRACTCE", right_on="TRACTCE"
)
requests_Gdf = requests_Gdf.merge(
    requests_counts, left_on="TRACTCE", right_on="TRACTCE"
)
requests_Gdf.head()

In [ ]:
# Creating the visualization of top request category by census tract

tooltip_cols2 = ["median_household_income", "poverty_rate"] + list(
    requests_counts.columns
)

request_source_map = requests_Gdf.explore(
    column="top request category",
    cmap="rainbow",
    tooltip=tooltip_cols2,
    legend=True,
    legend_kwds={"caption": "Request Category"},
)

title_request_source_map = """
     <h3 align="center" style="font-size:20px"><b>Top 311 Request Types by Census Tract, San Francisco 2023</b></h3>
"""

request_source_map.get_root().html.add_child(folium.Element(title_request_source_map))
request_source_map.save("figures/request_source_map.html")
request_source_map

In [ ]:
# Creating the income buckets for the stacked bar

income_bins = [0, 75000, 100000, 125000, 150000, 175000, 200000, float("inf")]
income_labels = [
    "<75K",
    "75K-100K",
    "100K-125K",
    "125K-150K",
    "150K-175K",
    "175K-200K",
    "200K+",
]
requests_Gdf["income_bucket"] = pd.cut(
    requests_Gdf["median_household_income"], bins=income_bins, labels=income_labels
)

In [ ]:
# Grouping and finding percentage splits of requests by income group

inc_grouped = requests_Gdf.groupby("income_bucket")[list(requests_counts.columns)].sum()
inc_grouped_pct = inc_grouped.div(inc_grouped.sum(axis=1), axis=0)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

inc_grouped_pct.plot(kind="bar", stacked=True, ax=ax)
plt.ylabel("Percentage of Requests")
plt.xlabel("Median Household Income Bracket")
plt.title("311 Request Type Percentages by Median Household Income Bracket")
plt.legend(title="Request Type", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.savefig("figures/request_type_stacked_bar.jpeg")
plt.show()

# Sidewalk and Street Cleaning Requests per Capita by Census Tract


Scatterplot of Median HH income and per capita requests

Median response time by census tract income


In [ ]:
# Filtering for the most common category of 311 requests, Street and Sidewalk Cleaning
# Cleaning and organizing data for mapping

street_clean_gdf = joined_Gdf[joined_Gdf["category"] == "Street And Sidewalk Cleaning"]
street_clean_gdf = street_clean_gdf[street_clean_gdf["status"] == "Closed"]
street_clean_gdf["open_close_timeframe"] = (
    street_clean_gdf["closed"] - street_clean_gdf["opened"]
)
street_clean_gdf["Month"] = street_clean_gdf["closed"].dt.to_period("M")
street_clean_gdf.head()

In [ ]:
annual_tract_counts = (
    street_clean_gdf.groupby(["TRACTCE"]).size().reset_index(name="request_count")
)
annual_tract_counts = annual_tract_counts.merge(
    sf_tracts_acs[
        [
            "TRACTCE",
            "total_population",
            "median_household_income",
            "poverty_rate",
            "geometry",
            "share_white",
            "share_black",
            "share_hispanic",
        ]
    ],
    on="TRACTCE",
    how="left",
)
annual_tract_counts = annual_tract_counts[annual_tract_counts["total_population"] > 0]
annual_tract_counts["req_per_capita"] = (
    100 * annual_tract_counts["request_count"] / annual_tract_counts["total_population"]
)
annual_counts_Gdf = gpd.GeoDataFrame(
    annual_tract_counts, geometry="geometry", crs=sf_tracts_acs.crs
)
annual_counts_Gdf = annual_counts_Gdf[annual_counts_Gdf["total_population"] > 1000]

In [ ]:
# Making map for sidewalk & street cleaning requests per 100 residents by census tract

tooltip_counts = [
    "req_per_capita",
    "total_population",
    "median_household_income",
    "poverty_rate",
]

ss_counts_map = annual_counts_Gdf.explore(
    column="req_per_capita",
    cmap="RdYlGn",
    tooltip=tooltip_counts,
    legend=True,
    legend_kwds={"caption": "Requests per 100 Residents"},
)

title_ss_counts_map = """
     <h3 align="center" style="font-size:20px"><b>Sidewalk & Street Cleaning Requests per 100 Residents by Census Tract</b></h3>
"""

ss_counts_map.get_root().html.add_child(folium.Element(title_ss_counts_map))
ss_counts_map.save("figures/ss_cleaning_map.html")
ss_counts_map

In [ ]:
# scatterplot to show the relationship between income and number of requests per capita

import numpy as np

fig, ax = plt.subplots(figsize=(10, 6))

x = annual_counts_Gdf["median_household_income"]
y = np.log(annual_counts_Gdf["req_per_capita"])

plt.scatter(x, y)

"""
m, b = np.polyfit(x, y, 1)
trend = m * x + b

plt.plot(x, trend)
"""

plt.ylabel("Log Requests per 100 Residents")
plt.xlabel("Census Tract Median Household Income (2023, ACS 5-year Rolling Survey)")
plt.title(
    "Sidewalk & Street Cleaning 311 Requests per 100 Residents by Tract Median HH Income, San Francisco 2023"
)
plt.tight_layout()
plt.savefig("figures/Cleaning_request_scatterplot.jpeg")
plt.show()

# Graveyard

Ideas in progress, to be potentially revisited later


In [ ]:
# Create the monthly slider for sidewalk & street cleaning map -- to be done later

"""
monthly_tract_counts = street_clean_gdf.groupby(['TRACTCE', 'Month']).size().reset_index(name='request_count')
monthly_tract_counts = monthly_tract_counts.merge(sf_tracts_acs[['TRACTCE', 'total_population', 'median_household_income', 'poverty_rate', 'geometry']], on = 'TRACTCE', how = 'left')
monthly_tract_counts = monthly_tract_counts[monthly_tract_counts['total_population'] > 0]
monthly_tract_counts['req_per_capita'] = 1000*monthly_tract_counts['request_count']/monthly_tract_counts['total_population']
monthly_counts_Gdf = gpd.GeoDataFrame(monthly_tract_counts, geometry='geometry', crs=sf_tracts_acs.crs)
monthly_counts_Gdf['month_str'] = monthly_counts_Gdf['Month'].astype(str)
monthly_counts_Gdf
"""

In [ ]:
# Examining the top sources of 311 requests across census tracts
"""source_counts = joined_Gdf.groupby(['TRACTCE', 'source']).size().unstack(fill_value = 0)
tract_source_total = source_counts.sum(axis = 1)
source_pct = 100*source_counts.div(tract_source_total, axis = 0)

top_source_tract = source_counts.idxmax(axis = 1)
top_intensity_tract = source_pct.max(axis = 1)

sources_Gdf = sf_tracts_acs
sources_Gdf = sources_Gdf.merge(top_intensity_tract.rename('top intensity'), left_on = 'TRACTCE', right_on = 'TRACTCE')
sources_Gdf = sources_Gdf.merge(top_source_tract.rename('top source'), left_on = 'TRACTCE', right_on = 'TRACTCE')
sources_Gdf = sources_Gdf.merge(source_counts, left_on = 'TRACTCE', right_on = 'TRACTCE')
sources_Gdf.head()
"""

In [ ]:
"""
source_colors = {
    'Mobile/Open311': 'Greens',
    'Phone': 'Blues',
    'Web': 'Yellows',
    'Integrated Agency': 'Reds'
}
"""

In [ ]:
"""
tooltip_cols = list(source_counts.columns) + ['top source', 'top intensity']

m = sources_Gdf.explore(
    column = 'top intensity',
    cmap = 'Blues',
    tooltip=tooltip_cols,
    legend = True,
)

m
"""